# Dataset Preparation

### M3CV database

#### 动态加载

In [ ]:
# !pip install librosa
# !pip install PyWavelets
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 9.7 MB/s eta 0:00:00:00:0100:01


In [ ]:
!pip uninstall protobuf
!pip install protobuf==3.20.0


In [1]:
import os
os.chdir('D:\EEG-ESRT\eeg-biometric-competition')

In [2]:
import numpy as np
import mne
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Reshape
import os
import numpy as np
import scipy.io as scio
import pandas as pd
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical


# 读取数据
train_images = pd.read_csv('Enrollment_Info.csv')
val_images = pd.read_csv('Calibration_Info.csv')

train_images = shuffle(train_images, random_state=0)
val_images = shuffle(val_images)

train_image_list = train_images
val_image_list = val_images

df = train_image_list
train_img_list = train_image_list['EpochID'].values
print(train_image_list.head())
train_label_list = train_image_list['subject'].values


val_img_list = val_image_list['EpochID'].values
val_label_list = val_image_list['SubjectID'].values
print(train_img_list,train_label_list)

           EpochID subject  session  condition  usage
44959  epoch044960  sub074        1          7      1
26978  epoch026979  sub045        1          3      1
35644  epoch035645  sub059        1         10      1
36966  epoch036967  sub061        1          8      1
50637  epoch050638  sub084        1         12      1
['epoch044960' 'epoch026979' 'epoch035645' ... 'epoch042614' 'epoch043568'
 'epoch002733'] ['sub074' 'sub045' 'sub059' ... 'sub070' 'sub072' 'sub005']


In [3]:
import numpy as np
import pywt

def wavelet_transform(x):
    coeffs = pywt.dwt(x, wavelet='haar')
    return coeffs

# 创建一个示例 65x1000 的脑电信号数据
data = np.random.rand(65, 1000)

# 对数据进行小波变换
coeffs_list = [wavelet_transform(channel_data) for channel_data in data]

# 提取高频和低频系数
high_freq_coeffs = np.array([coeffs[0] for coeffs in coeffs_list])
low_freq_coeffs = np.array([coeffs[1] for coeffs in coeffs_list])

# 再次进行小波变换
high_freq_coeffs_list = [wavelet_transform(channel_data) for channel_data in high_freq_coeffs]
low_freq_coeffs_list = [wavelet_transform(channel_data) for channel_data in low_freq_coeffs]

# 提取二次小波变换后的高频和低频系数
high_freq_coeffs_1 = np.array([coeffs[0] for coeffs in high_freq_coeffs_list])
high_freq_coeffs_2 = np.array([coeffs[1] for coeffs in high_freq_coeffs_list])
low_freq_coeffs_1 = np.array([coeffs[0] for coeffs in low_freq_coeffs_list])
low_freq_coeffs_2 = np.array([coeffs[1] for coeffs in low_freq_coeffs_list])

# 将四个 65x250 的系数堆叠在一起，得到 (4*65)x250 的矩阵
final_data = np.concatenate((high_freq_coeffs_1, high_freq_coeffs_2, low_freq_coeffs_1, low_freq_coeffs_2), axis=0)

print("Original shape:", data.shape)
print("Final data shape:", final_data.shape)

Original shape: (65, 1000)
Final data shape: (260, 250)


In [4]:
import numpy as np
import librosa

def transform(data, n_mfcc=13, hop_length=15, n_fft=60):
    # 初始化一个空的 MFCC 矩阵
    mfcc_matrix = np.empty((0,))

    # 遍历每个通道，并计算 MFCC
    for channel_data in data:
        channel_mfcc = librosa.feature.mfcc(y=channel_data, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
        print("Channel MFCC shape:", channel_mfcc.shape)

        # 展平 channel_mfcc 并添加到 mfcc_matrix
        mfcc_matrix = np.hstack([mfcc_matrix, channel_mfcc.flatten()])

    print("Flattened MFCC shape:", mfcc_matrix.shape)

    # 计算需要填充的元素数量以使元素总数可以被 224 整除
    padding_size = int(np.ceil(mfcc_matrix.size / 224) * 224) - mfcc_matrix.size

    # 使用零填充 mfcc_matrix
    padded_mfcc_matrix = np.pad(mfcc_matrix, (0, padding_size), mode='constant', constant_values=0)
    print("Padded MFCC shape:", padded_mfcc_matrix.shape)

    # 将填充后的 mfcc_matrix 转换为形状为 (-1, 224) 的矩阵
    reshaped_data = np.reshape(padded_mfcc_matrix, (-1, 224))

    return reshaped_data

# 创建一个示例 65x1000 的脑电信号数据
data = np.random.rand(65, 1000)

# 将数据转换为 MFCC 矩阵
transformed_data = transform(data)

print("Original shape:", data.shape)
print("Transformed shape:", transformed_data.shape)

ModuleNotFoundError: ignored

In [ ]:
import numpy as np
import pywt
import librosa
import cv2

def wavelet_transform(x):
    coeffs = pywt.dwt(x, wavelet='haar')
    return coeffs

def process_coeffs(coeffs, target_shape=(224, 224)):
    resized_data = cv2.resize(coeffs, target_shape)
    return resized_data

def transform_mfcc(data, n_mfcc=13, hop_length=15, n_fft=60):
    # 初始化一个空的 MFCC 矩阵
    mfcc_matrix = np.empty((0,))

    # 遍历每个通道，并计算 MFCC
    for channel_data in data:
        channel_mfcc = librosa.feature.mfcc(y=channel_data, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

        # 展平 channel_mfcc 并添加到 mfcc_matrix
        mfcc_matrix = np.hstack([mfcc_matrix, channel_mfcc.flatten()])

    # 计算需要填充的元素数量以使元素总数可以被 224 整除
    padding_size = int(np.ceil(mfcc_matrix.size / 224) * 224) - mfcc_matrix.size

    # 使用零填充 mfcc_matrix
    padded_mfcc_matrix = np.pad(mfcc_matrix, (0, padding_size), mode='constant', constant_values=0)

    # 将填充后的 mfcc_matrix 转换为形状为 (-1, 224) 的矩阵
    reshaped_data = np.reshape(padded_mfcc_matrix, (-1, 224))
    return reshaped_data




def transform(data):
    # 对数据进行小波变换
    coeffs_list = [wavelet_transform(channel_data) for channel_data in data]

    # 提取高频和低频系数
    high_freq_coeffs = np.array([coeffs[0] for coeffs in coeffs_list])
    low_freq_coeffs = np.array([coeffs[1] for coeffs in coeffs_list])

    # 再次进行小波变换
    high_freq_coeffs_list = [wavelet_transform(channel_data) for channel_data in high_freq_coeffs]
    low_freq_coeffs_list = [wavelet_transform(channel_data) for channel_data in low_freq_coeffs]

    # 提取二次小波变换后的高频和低频系数
    high_freq_coeffs_1 = np.array([coeffs[0] for coeffs in high_freq_coeffs_list])
    high_freq_coeffs_2 = np.array([coeffs[1] for coeffs in high_freq_coeffs_list])
    low_freq_coeffs_1 = np.array([coeffs[0] for coeffs in low_freq_coeffs_list])
    low_freq_coeffs_2 = np.array([coeffs[1] for coeffs in low_freq_coeffs_list])

    # 将两个高频系数矩阵连接在一起，形成130x250的矩阵
    high_freq_matrix = np.concatenate((high_freq_coeffs_1, high_freq_coeffs_2), axis=0)

    # 将两个低频系数矩阵连接在一起，形成130x250的矩阵
    low_freq_matrix = np.concatenate((low_freq_coeffs_1, low_freq_coeffs_2), axis=0)

    # 将130x250的高频和低频矩阵调整为224x224的图像
    high_freq_image = process_coeffs(high_freq_matrix)
    low_freq_image = process_coeffs(low_freq_matrix)

    # 计算 MFCC
    mfcc_data = transform_mfcc(data)

    # 将 MFCC 数据处理为224x224的图像
    mfcc_image = process_coeffs(mfcc_data)

    # 将三层图像堆叠在一起得到 224x224x3 的图像
    final_image = np.stack([high_freq_image, low_freq_image, mfcc_image], axis=2)

    return final_image

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import scipy.io as scio
from tensorflow.data import Dataset
from tqdm import tqdm


class MyDataset:
    def __init__(self, train_img_list, val_img_list, train_label_list, val_label_list, mode='train', n_samples=10):
        self.mode = mode
        self.train_images = train_img_list[:n_samples]
        self.test_images = val_img_list[:n_samples]
        self.train_label = train_label_list[:n_samples]
        self.test_label = val_label_list[:n_samples]

    def load_eeg(self, eeg_path):
        data = scio.loadmat(eeg_path)
        return data['epoch_data']

    def __len__(self):
        if self.mode == 'train':
            return len(self.train_images)
        else:
            return len(self.test_images)
    def get_data_generator(self):
        if self.mode == 'train':
            image_list = self.train_images
            label_list = self.train_label
        else:
            image_list = self.test_images
            label_list = self.test_label
        for img, la in zip(image_list, label_list):
            if self.mode == 'train':
                eeg_path = os.path.join('/home/u200810216/jupyter/kaggle_m3cv/train/', img + '.mat')
            else:
                eeg_path = os.path.join('/home/u200810216/jupyter/kaggle_m3cv/val/', img + '.mat')

            data = self.load_eeg(eeg_path)

            data = transform(data)


#             data = data[..., np.newaxis]
            label = to_categorical((int(la[4:])-1), 95)
            yield data, label

n_samples= None
# 创建MyDataset实例
train_dataset = MyDataset(train_img_list, val_img_list, train_label_list, val_label_list, mode='train', n_samples=n_samples)
val_dataset = MyDataset(train_img_list, val_img_list, train_label_list, val_label_list, mode='test', n_samples=n_samples)




In [ ]:
import os
import numpy as np
import scipy.io as scio

def load_eeg(eeg_path):
    data = scio.loadmat(eeg_path)
    return data['epoch_data']

def transform_and_save_data(images_list, images_dir, save_dir):
    for img in tqdm(images_list):
        eeg_path = os.path.join(images_dir, img + '.mat')
        data = load_eeg(eeg_path)

        data = transform(data)  # 假设你有一个transform函数来进行数据预处理

        np.save(os.path.join(save_dir, img), data)  # 保存预处理过的数据为.npy文件

# 为训练集和验证集进行预处理
train_images_dir = '/home/u200810216/jupyter/kaggle_m3cv/train/'
val_images_dir = '/home/u200810216/jupyter/kaggle_m3cv/val/'

save_train_dir = '/home/u200810216/jupyter/kaggle_m3cv/train_npy/'
save_val_dir = '/home/u200810216/jupyter/kaggle_m3cv/val_npy/'

# 创建保存的文件夹
os.makedirs(save_train_dir, exist_ok=True)
os.makedirs(save_val_dir, exist_ok=True)

# 执行预处理并保存
transform_and_save_data(train_img_list, train_images_dir, save_train_dir)
transform_and_save_data(val_img_list, val_images_dir, save_val_dir)


100%|██████████| 12168/12168 [27:11<00:00,  7.46it/s]


In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.data import Dataset

class MyDataset:
    def __init__(self, train_img_list, val_img_list, train_label_list, val_label_list, mode='train', n_samples=10):
        self.mode = mode
        self.train_images = train_img_list[:n_samples]
        self.test_images = val_img_list[:n_samples]
        self.train_label = train_label_list[:n_samples]
        self.test_label = val_label_list[:n_samples]

    def load_data(self, data_path):
        data = np.load(data_path)
        return data

    def __len__(self):
        if self.mode == 'train':
            return len(self.train_images)
        else:
            return len(self.test_images)

    def get_data_generator(self):
        if self.mode == 'train':
            image_list = self.train_images
            label_list = self.train_label
            data_dir = '/home/u200810216/jupyter/kaggle_m3cv/train_npy/'
        else:
            image_list = self.test_images
            label_list = self.test_label
            data_dir = '/home/u200810216/jupyter/kaggle_m3cv/val_npy/'

        for img, la in zip(image_list, label_list):
            data_path = os.path.join(data_dir, img + '.npy')
            data = self.load_data(data_path)

            label = to_categorical((int(la[4:])-1), 95)
            yield data, label

n_samples= None
# 创建MyDataset实例
train_dataset = MyDataset(train_img_list, val_img_list, train_label_list, val_label_list, mode='train', n_samples=n_samples)
val_dataset = MyDataset(train_img_list, val_img_list, train_label_list, val_label_list, mode='test', n_samples=n_samples)


In [ ]:
# train_data = Dataset.from_generator(
#     train_dataset.get_data_generator,
#     output_signature=(
#         tf.TensorSpec(shape=(65, 1000,1), dtype=tf.float32),
#         tf.TensorSpec(shape=(95), dtype=tf.int32)
#     )
# )
# val_data = Dataset.from_generator(
#     val_dataset.get_data_generator,
#     output_signature=(
#         tf.TensorSpec(shape=(65, 1000,1), dtype=tf.float32),
#         tf.TensorSpec(shape=(95), dtype=tf.int32)
#     )
# )
train_data = Dataset.from_generator(
    train_dataset.get_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(95), dtype=tf.int32)
    )
)
val_data = Dataset.from_generator(
    val_dataset.get_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(95), dtype=tf.int32)
    )
)


2023-06-03 01:46:53.435907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 01:46:56.054392: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38400 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:51:00.0, compute capability: 8.0


In [ ]:
# train_data = train_data.shuffle(buffer_size=len(train_dataset)).batch(256)
train_data = train_data.batch(128)
val_data = val_data.batch(128)
val_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 95), dtype=tf.int32, name=None))>

# Network Architecture

## Bert-Vision Transfomer

In [ ]:
import os
import tensorflow as tf
from transformers import TFViTModel, ViTConfig
from tensorflow.keras.callbacks import ModelCheckpoint

# 加载预先训练好的 ViT 模型的配置
config = ViTConfig.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=95)

# 修改配置，使其适用于单通道图像分类任务
config.input_channels = 3
config.hidden_dropout_prob = 0.0
config.attention_probs_dropout_prob = 0.0

In [ ]:
from transformers import create_optimizer
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [ ]:
batch_size = 16
epochs = 5
num_train_steps = len(train_img_list) * epochs
learning_rate = 3e-5
weight_decay_rate = 0.01

vit = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k", config=config)

# # 使用单通道图像进行分类
# inputs = tf.keras.layers.Input(shape=(224, 224, 3))# 这里等待修改
# x = tf.image.resize(inputs, (224, 224))
# x = tf.keras.layers.Concatenate(axis=-1)([x, x, x])
# x = tf.keras.layers.Lambda(lambda x: tf.transpose(x, [0, 3, 1, 2]))(x)
# outputs = vit(x).last_hidden_state[:, 0, :]


# 使用单通道图像进行分类
inputs = tf.keras.layers.Input(shape=(224, 224, 3))# 这里等待修改
x = tf.keras.layers.Lambda(lambda x: tf.transpose(x, [0, 3, 1, 2]))(inputs)
outputs = vit(x).last_hidden_state[:, 0, :]

# 添加输出层
outputs = tf.keras.layers.Dense(95, activation='softmax')(outputs)
# outputs = tf.keras.layers.Lambda(lambda x: tf.argmax(x, axis=1))(outputs)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=1,
)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
# loss = tf.keras.losses.SparseCategoricalCrossentropy()


model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy',
                       'FalsePositives',
                       'FalseNegatives'])


# 记录日志
log_dir = "logs/transfomer"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# 定义一个回调函数来保存模型和历史记录
checkpoint_callback = ModelCheckpoint(
    filepath='ViT-3-layer.h5',
    save_weights_only=True,
    save_best_only=False,
    monitor='val_loss',
    verbose=1,
)


2023-06-03 01:47:50.223109: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-06-03 01:47:51.052279: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-03 01:47:57.803909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [ ]:
len(train_img_list)

In [ ]:
# 训练模型
model.load_weights('/home/u200810216/jupyter/kaggle_m3cv/ViT-3-layer.h5')
history = model.fit(val_data,
                    epochs=5,
                    verbose=1,
                    validation_data=val_data,
                    callbacks=[tensorboard_callback, checkpoint_callback])
# 保存模型权重而不是整个模型
model.save_weights('/home/u200810216/jupyter/kaggle_m3cv/ViT-3layer-val.h5')

Epoch 1/5
     96/Unknown - 328s 3s/step - loss: 1.1199 - accuracy: 0.7219 - false_positives: 1776.0000 - false_negatives: 4325.0000
Epoch 1: saving model to ViT-3-layer.h5
96/96 [==============================] - 356s 4s/step - loss: 1.1199 - accuracy: 0.7219 - false_positives: 1776.0000 - false_negatives: 4325.0000 - val_loss: 0.3185 - val_accuracy: 0.9053 - val_false_positives: 663.0000 - val_false_negatives: 1586.0000
Epoch 2/5
96/96 [==============================] - ETA: 0s - loss: 0.2464 - accuracy: 0.9280 - false_positives: 577.0000 - false_negatives: 1143.0000
Epoch 2: saving model to ViT-3-layer.h5
96/96 [==============================] - 69s 714ms/step - loss: 0.2464 - accuracy: 0.9280 - false_positives: 577.0000 - false_negatives: 1143.0000 - val_loss: 0.1327 - val_accuracy: 0.9652 - val_false_positives: 293.0000 - val_false_negatives: 581.0000
Epoch 3/5
96/96 [==============================] - ETA: 0s - loss: 0.1036 - accuracy: 0.9726 - false_positives: 229.0000 - false_ne

In [ ]:
# 训练模型
history = model.fit(train_data,
                    epochs=25,
                    verbose=1,
                    validation_data=train_data,
                    callbacks=[tensorboard_callback, checkpoint_callback])
# 保存模型权重而不是整个模型
model.save_weights('/home/u200810216/jupyter/kaggle_m3cv/ViT-3layer-epoch62.h5')

Epoch 1/25
    451/Unknown - 956s 2s/step - loss: 1.1704e-04 - accuracy: 1.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00

In [ ]:
model.save_weights('/home/u200810216/jupyter/kaggle_m3cv/ViT-3layer-epoch12.h5')

In [ ]:

history = model.fit(val_data,
                    epochs=1,
                    verbose=1,
                    validation_data=val_data,
                    callbacks=[tensorboard_callback, checkpoint_callback])
model.save_weights('/home/u200810216/jupyter/kaggle_m3cv/ViT-3layer-epoch12-val1.h5')

     96/Unknown - 221s 2s/step - loss: 0.8837 - accuracy: 0.7643 - false_positives: 1116.0000 - false_negatives: 4220.0000
Epoch 1: saving model to ViT-3-layer.h5
96/96 [==============================] - 248s 3s/step - loss: 0.8837 - accuracy: 0.7643 - false_positives: 1116.0000 - false_negatives: 4220.0000 - val_loss: 0.3378 - val_accuracy: 0.9012 - val_false_positives: 686.0000 - val_false_negatives: 1650.0000


In [ ]:
model.load_weights('/home/u200810216/jupyter/kaggle_m3cv/vision_transfomer_test_kaggle_batch128_weights_val_3.h5')
history = model.fit(train_data,
                    epochs=3,
                    verbose=1,
                    validation_data=val_data,
                    callbacks=[tensorboard_callback, checkpoint_callback])

# 保存模型
model.save_weights('/home/u200810216/jupyter/kaggle_m3cv/vision_transfomer_test_kaggle_batch128_weights_val_3_train_3.h5')  # 保存为 .h5 文件格式


Epoch 1/3
    452/Unknown - 1503s 3s/step - loss: 0.0604 - accuracy: 0.9852 - false_positives: 614.0000 - false_negatives: 1015.0000
Epoch 1: saving model to transfomer.h5
452/452 [==============================] - 1755s 4s/step - loss: 0.0604 - accuracy: 0.9852 - false_positives: 614.0000 - false_negatives: 1015.0000 - val_loss: 2.9653 - val_accuracy: 0.4057 - val_false_positives: 5381.0000 - val_false_negatives: 7663.0000
Epoch 2/3
452/452 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 1.0000 - false_positives: 2.0000 - false_negatives: 2.0000
Epoch 2: saving model to transfomer.h5
452/452 [==============================] - 1017s 2s/step - loss: 0.0021 - accuracy: 1.0000 - false_positives: 2.0000 - false_negatives: 2.0000 - val_loss: 3.0420 - val_accuracy: 0.4031 - val_false_positives: 5474.0000 - val_false_negatives: 7679.0000
Epoch 3/3
452/452 [==============================] - ETA: 0s - loss: 0.0011 - accuracy: 1.0000 - false_positives: 0.0000e+00 - false_ne

In [ ]:
import os
epochs = 64

weights_path = '/home/u200810216/jupyter/kaggle_m3cv/'
initial_weights_file = 'vision_transfomer_test_kaggle_batch128_weights_val_3_train_3_roll_6.h5'
final_weights_file = f'vision_transfomer_test_kaggle_batch128_weights_val_3_train_3_roll_6_roll{epochs}.h5'

# 加载预训练权重
model.load_weights(os.path.join(weights_path, initial_weights_file))

# 训练循环
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    if epoch % 2 == 0:
        print("Training on train_data")
        data = train_data
    else:
        print("Training on val_data")
        data = val_data

    history = model.fit(data,
                        epochs=1,
                        verbose=1,
                        callbacks=[tensorboard_callback, checkpoint_callback])

    # 保存模型参数
    model.save_weights(os.path.join(weights_path, "{}_epoch_{}.h5".format(final_weights_file.replace(".h5",""), epoch+1)))

# 保存最终模型参数
model.save_weights(os.path.join(weights_path, final_weights_file))


Epoch 1/64
Training on train_data
    452/Unknown - 1362s 3s/step - loss: 0.0280 - accuracy: 0.9936 - false_positives: 261.0000 - false_negatives: 500.0000
Epoch 1: saving model to transfomer.h5
452/452 [==============================] - 1369s 3s/step - loss: 0.0280 - accuracy: 0.9936 - false_positives: 261.0000 - false_negatives: 500.0000
Epoch 2/64
Training on val_data
     96/Unknown - 185s 2s/step - loss: 0.1188 - accuracy: 0.9651 - false_positives: 297.0000 - false_negatives: 555.0000
Epoch 1: saving model to transfomer.h5
96/96 [==============================] - 191s 2s/step - loss: 0.1188 - accuracy: 0.9651 - false_positives: 297.0000 - false_negatives: 555.0000
Epoch 3/64
Training on train_data
    452/Unknown - 716s 2s/step - loss: 0.0218 - accuracy: 0.9956 - false_positives: 191.0000 - false_negatives: 344.0000
Epoch 1: saving model to transfomer.h5
452/452 [==============================] - 722s 2s/step - loss: 0.0218 - accuracy: 0.9956 - false_positives: 191.0000 - false_ne

## 结果预测与提交

In [ ]:
def transform_and_save_test_data(test_img_list, test_images_dir, save_dir):
    for img in tqdm(test_img_list):
        eeg_path = os.path.join(test_images_dir, img + '.mat')
        data = load_eeg(eeg_path)

        data = transform(data)  # 假设你有一个transform函数来进行数据预处理

        np.save(os.path.join(save_dir, img), data)  # 保存预处理过的数据为.npy文件

test_images_dir = '/home/u200810216/jupyter/kaggle_m3cv/test/'
save_test_dir = '/home/u200810216/jupyter/kaggle_m3cv/test_npy/'
# Read the test data
test_image = pd.read_csv('/home/u200810216/jupyter/kaggle_m3cv/Testing_Info.csv')
test_image_path_list = test_image['Row'].values


# 创建保存的文件夹
os.makedirs(save_test_dir, exist_ok=True)

# 执行预处理并保存
transform_and_save_test_data(test_image_path_list, test_images_dir, save_test_dir)


100%|██████████| 52342/52342 [2:00:04<00:00,  7.26it/s]  


In [ ]:
from tqdm import tqdm


def load_data(data_path):
    data = np.load(data_path)
    return data

batch_size = 512

pre_list = []
# Read the test data
test_image = pd.read_csv('/home/u200810216/jupyter/kaggle_m3cv/Testing_Info.csv')
test_image_path_list = test_image['Row'].values
data_dir = '/home/u200810216/jupyter/kaggle_m3cv/test_npy/'
data_list = [os.path.join(data_dir, img + '.npy') for img in test_image_path_list]

for i in tqdm(range(0, len(data_list), batch_size)):
    batch_data_list = data_list[i:i+batch_size]
    batch_data = np.array([load_data(data_path) for data_path in batch_data_list]).astype('float32')
    batch_data = batch_data[..., np.newaxis]

    batch_data_tensor = tf.convert_to_tensor(batch_data)
    batch_out = model(batch_data_tensor)
    batch_pre_list = np.argmax(batch_out.numpy(), axis=-1) + 1

    pre_list.extend(batch_pre_list)

# 以下的后处理和保存结果的代码可以保持不变。
# Post-processing and saving results
img_test = pd.DataFrame(test_image_path_list, columns=["EpochID"])
img_pre = pd.DataFrame(test_image_path_list, columns=["EpochID"])
img_pre['SubjectID'] = pre_list
pre_info = img_pre['SubjectID'].values
test_info = test_image['SubjectID'].values

result_cnn = list()

for i, j in zip(test_info, pre_info):
    if i == 'None':
        result_cnn.append(j)
    elif int(i[4:])==j :
        print(i[4:])
        result_cnn.append(int(1))
    else:
        result_cnn.append(int(0))

img_test['Prediction'] = result_cnn
img_test.to_csv('result_vit_kaggle_finetuning_37_5.csv', index=False)

100%|██████████| 103/103 [22:39<00:00, 13.20s/it]


87
71
33
39
76
32
73
95
39
51
71
33
83
33
87
84
39
71
33
51
33
33
39
71
32
45
33
33
39
57
71
62
58
73
57
73
32
24
76
39
31
76
33
76
39
76
39
53
87
78
53
76
51
21
65
51
83
76
39
52
44
71
